# Imports

In [ ]:
from fastbook import *


# Gethin images

In [ ]:
# generatin heros list
def imgDownload(imgList = ['batman','flash'], imgNum = 3,folderName = 'imgs'):
    path = Path(folderName) # path to images
    if not path.exists(): # if path doesn't exist
        path.mkdir()    # create path
        for o in imgList:   # for each hero in list
            dest = Path(path/o) # create path to image
            dest.mkdir(exist_ok=True) # create path for hero
            urls = search_images_ddg(o,max_images= imgNum) # get urls for hero
            for index,url in enumerate(urls): # for each url
                download_url(url,f"{dest}/{index}.jpg") # download image
    return path
        

In [ ]:
# checking integrity of images
def imgCheck(path):
    fns = get_image_files(path) # get all image files
    failed = verify_images(fns) # check if images are valid
    failed.map(Path.unlink) # delete images that are not valid
    return len(failed)


# generating DataLoader

In [ ]:
dlGenerator = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # ImageBlock is the image block
    get_items=get_image_files, # get all image files    
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # split into train and valid
    get_y=parent_label, # get parent label
    item_tfms= RandomResizedCrop(224,min_scale = 0.5), # resize images to 128x128
    batch_tfms= aug_transforms(mult = 2), # augment images
)

# Generating and training the model

In [ ]:
path = imgDownload(imgNum = 150) # download images
imgCheck(path)  # check images
learn = cnn_learner(pepe, models.resnet34, metrics=accuracy) # create learner
learn.fine_tune(5) # fine tune model

# Inspecting results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(5, nrows=1)

# Cleaning Dataset

cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# Delete images taged in the cleaner
for idx in cleaner.delete(): cleaner.fns[idx].unlink()